In [1]:
import os

import numpy as np
import pandas as pd
import random

from agents.matchmanager import buildMatchManager

from multiprocessing import Pool

In [2]:
agents = ['RandomAgent','AlphaBetaAgent','GreedyAgent']
scenarios = ['scenarioTest1v1', 'scenarioTest2v2', 'scenarioJunctionExo', 'scenarioJunction', 'scenarioRoadblock', 'scenarioBridgeHead', 'scenarioCrossingTheCity']

In [3]:
def game(args):
    scenario, red, blue, seed = args
    vectors=[]

    mm = buildMatchManager('', scenario, red, blue, seed=seed)
    
    while not mm.end:
        mm.nextStep()
        v = list(mm.state.vector())
        vectors.append(v)
    
    for v in vectors:
        v += [mm.winner, scenario, red, blue, seed]
    
    cols = list(mm.state.vectorInfo()) + ['winner', 'meta_scenario', 'meta_p_red', 'meta_p_blue', 'meta_seed']
        
    df = pd.DataFrame(data=vectors, columns=cols)
    df.to_pickle(f'data/{scenario}.{red}.{blue}.{seed}.pkl.gz', compression='gzip')


In [4]:
game(('scenarioTest1v1', 'GreedyAgent', 'GreedyAgent', 1))

In [5]:
args = []

np.random.seed(42)

for _ in range(10000):
    scenario = np.random.choice(scenarios)
    red = np.random.choice(agents)
    blue = np.random.choice(agents)
    seed = np.random.randint(100000000, 999999999)
    
    args.append((scenario, red, blue, seed))

In [6]:
args

[('scenarioCrossingTheCity', 'RandomAgent', 'GreedyAgent', 227521863),
 ('scenarioRoadblock', 'RandomAgent', 'GreedyAgent', 941095289),
 ('scenarioJunctionExo', 'GreedyAgent', 'GreedyAgent', 998717130),
 ('scenarioRoadblock', 'GreedyAgent', 'AlphaBetaAgent', 342285876),
 ('scenarioTest2v2', 'AlphaBetaAgent', 'AlphaBetaAgent', 103344769),
 ('scenarioJunction', 'RandomAgent', 'RandomAgent', 332968651),
 ('scenarioTest2v2', 'AlphaBetaAgent', 'RandomAgent', 881447915),
 ('scenarioTest1v1', 'RandomAgent', 'GreedyAgent', 580404538),
 ('scenarioCrossingTheCity', 'AlphaBetaAgent', 'GreedyAgent', 985063869),
 ('scenarioBridgeHead', 'GreedyAgent', 'AlphaBetaAgent', 957592370),
 ('scenarioJunction', 'GreedyAgent', 'RandomAgent', 299502978),
 ('scenarioRoadblock', 'GreedyAgent', 'GreedyAgent', 832395540),
 ('scenarioTest1v1', 'GreedyAgent', 'AlphaBetaAgent', 954206851),
 ('scenarioTest1v1', 'AlphaBetaAgent', 'AlphaBetaAgent', 681609465),
 ('scenarioTest1v1', 'AlphaBetaAgent', 'RandomAgent', 891274

In [ ]:
with Pool(64) as p:
    p.map(game, args)

In [ ]:
def compress(scenario):
    
    files = [f for f in os.listdir('data') if scenario in f]
    
    dfs = [pd.concat([pd.read_pickle(os.path.join('data', f), compression='gzip') for f in files[i:i+1000]]) for i in range(0, len(files), 1000)]
    
    dfs = pd.concat(dfs)
    
    dfs.to_pickle(f'data.{scenario}.pkl.gz', compression='gzip')

In [ ]:
with Pool(len(scenarios)) as p:
    p.map(compress, scenarios)

In [ ]:
df = pd.read_pickle('data.scenarioJunction.pkl.gz', compression='gzip')

In [ ]:
len(sorted(list(df.columns)))

In [ ]:
for c in df.columns:
    print(c)